# A basic training loop

## From the last notebook...

In [134]:
#export
import pickle, gzip, torch, math, numpy as np, torch.nn.functional as F
from pathlib import Path
from IPython.core.debugger import set_trace
from torch import nn, optim

In [236]:
#export
from torch.utils.data import TensorDataset, DataLoader, Dataset
from dataclasses import dataclass
from typing import Any, Collection, Callable
from functools import partial, reduce

**Load the MNIST training images and labels**

The data is downloaded in notebook *_001a_nn_basics* so make sure you have run through that notebook first.

We print out the min and max of the features to get a feel for the range of feature values. In the case of the MNIST dataset, the features for a specific training example correspond to pixel values that, as we can see, range from 0 to ~1.

In [3]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'mnist'

with gzip.open(PATH/'mnist.pkl.gz', 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

x_train,y_train,x_valid,y_valid = map(torch.tensor, (x_train,y_train,x_valid,y_valid))
x_train.min(),x_train.max()

(tensor(0.), tensor(0.9961))

In [4]:
torch.tensor(x_train)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

**Set the batch size, number of epochs and learning rate**

The fast.ai library uses lots of abbreviations so as to make the code more concise. A reference for abbreviations can be found [here](https://github.com/fastai/fastai_v1/blob/master/docs/abbr.md).

In [5]:
bs=64
epochs = 2
lr=0.2

**Load the training and validation datasets**

TensorDataset is a Pytorch utility class that implements the Pytorch Dataset API for a given list of tensors.

The Dataset API allows iteration, indexing and slicing along the first dimension of each tensor passed in.

In this case we are passing in two tensors: training features and classification targets.
So each iteration of the TensorDataset will return a tuple of length two with the following form: (x_features, y_target)

In case you aren't already familiar with the Pytorch Dataset class, [this tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) talks about the Dataset abstract class, how to use it, and how to implement custom datasets for new kinds of data.

In [6]:
train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)

In [7]:
#export
def loss_batch(model, xb, yb, loss_fn, opt=None):
    loss = loss_fn(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [8]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [9]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func
        
    def forward(self, x): return self.func(x)

## Simplify nn.Sequential layers

**Function Composition** 

Function composition is a great way to capture and parameterize common operations as a single concept. You can see this with the *PoolFlatten* function below.

The *Lambda* layer we created in the last notebook makes it easy to quickly create Pytorch layers for different purposes.

In [10]:
#export
def ResizeBatch(*size): return Lambda(lambda x: x.view((-1,)+size))
def Flatten(): return Lambda(lambda x: x.view((x.size(0), -1)))
def PoolFlatten(): return nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten())

**Define the model**

Thanks to our named Pytorch nn.Modules above, the meaning and intention of each of the layers in our model is clearer and less prone to error when we make changes.

On kernel size: We will nearly always use a kernel size of 3. **TODO: WHY?**

Next we have a stripped down CNN example:

In [11]:
model = nn.Sequential(
    ResizeBatch(1,28,28),
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

**Define a *get_data* function**

It's often conventient to define a *get_data* function that encapsulates the work of setting up the training, validation and sometimes test data. Parameterizing *get_data* makes it easy to do things like change the batch size, etc.

Notice in this scenario that we shuffle the training dataloader but not the validation dataloader. We want the validation loss to be calculated the same way every time so that we can tell if we are still learning and not overfitting. Shuffling the training data helps prevent overfitting when calculating the gradients to be applied after each batch.

In [12]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
            DataLoader(valid_ds, batch_size=bs*2))

train_dl,valid_dl = get_data(train_ds, valid_ds, bs)

**Set loss function**

[Here](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/) is tutorial explaining why cross entropy is a resonable loss function for classifciation tasks.

In [13]:
loss_fn = F.cross_entropy

**Set optimizer**

We stick with stochastic gradient descent as our optimizer. **TODO: Why?**

In [14]:
opt = optim.SGD(model.parameters(), lr=lr)

**Test our loss function**

We try out our loss function on one batch of X features and y targets to make sure it's working correctly.

In [15]:
loss_fn(model(x_valid[0:bs]), y_valid[0:bs])

tensor(2.3030, grad_fn=<NllLossBackward>)

**Fit**

Everything looks ready, we call the fit function we developed earlier for two epochs to confirm that the model learns.

In [16]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 1.019653450012207
1 0.482433015871048


## Transformations

We are going to refactor some of the data transformations out of the network and into a pipeline that is applied to the data being fed into the Dataloders.

This is more flexible, simplifies the model, and will be useful later when we want to apply additional transformations for things like data augmentation.

**Define transformations**

In this example our only transformation will be *mnist2image*. This is a utility function to reshape our features into 28x28 arrays.

X is a batch of features where the first dimension is the number of samples in the batch and the remaining dimensions define the shape of the training example. y is the target variable to be learned, in this case it an integer representing one of 10 image classes.

With MNIST data, the X features start out as a 1x784 vector and we want to convert the features to 1x28x28 images (see line 62). This helper function does that for an entire batch of features.

In [56]:
def mnist2image(b): return b.view(1,28,28)

In [192]:
#export
@dataclass
class TfmDataset(Dataset):
    ds: Dataset
    tfm: Callable = None
        
    def __len__(self): return len(self.ds)
    
    def __getitem__(self,idx):
        x,y = self.ds[idx]
        if self.tfm is not None: x = self.tfm(x)
        return x,y

In [58]:
train_tds = TfmDataset(train_ds, mnist2image)
valid_tds = TfmDataset(valid_ds, mnist2image)

In [59]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_tds, bs,   shuffle=True),
            DataLoader(valid_tds, bs*2, shuffle=False))

In [60]:
train_dl,valid_dl = get_data(train_ds, valid_ds, bs)

We make some checks to make sure that *mnist2image* is working correctly:
1. The input and output shapes are as expected
2. The input and output data (features) are the same

In [61]:
x,y = next(iter(valid_dl))

In [62]:
valid_ds[0][0].shape, x[0].shape

(torch.Size([784]), torch.Size([1, 28, 28]))

In [63]:
torch.allclose(valid_ds[0][0], x[0].view(-1))

True

## Refactor network

**Define layer types and loop over them**

When use a layer type more than once in a contiguous fashion (one after the other), it makes sense to define a function for that layer type and then use that function to build our model function. 

That is what we do here with *conv2_relu* with which we avoid the three subsequent lines of code in line 12 (this saving becomes more significant in deeper networks).

In [65]:
#export
def conv2_relu(nif, nof, ks, stride):
    return nn.Sequential(nn.Conv2d(nif, nof, ks, stride, padding=ks//2), nn.ReLU())

def simple_cnn(actns, kernel_szs, strides):
    layers = [conv2_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(PoolFlatten())
    return nn.Sequential(*layers)

In [66]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
    return model, optim.SGD(model.parameters(), lr=lr)

In [67]:
model,opt = get_model()

In [68]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 1.2896591108322144
1 0.624457543182373


## CUDA

**Run in GPU and add progress bar**

To run our Pytorch networks in the GPU we have to specify it in the code. This is done by setting *torch.device('cuda')*. We will also add a progress bar to keep track of the progress during training. This we acomplish with the *tqdm_notebook* module of the [tqdm](https://github.com/tqdm/tqdm) package.

We integrate both these features into a custom Dataloader which we build on top of the Pytorch Dataloader.

In [237]:
#export
from tqdm import tqdm, tqdm_notebook, trange, tnrange
from ipykernel.kernelapp import IPKernelApp

def in_notebook(): return IPKernelApp.initialized()

def to_device(device, b): return [o.to(device) for o in b]
default_device = torch.device('cuda')

if in_notebook():
    tqdm = tqdm_notebook
    trange = tnrange

@dataclass
class DeviceDataLoader():
    dl: DataLoader
    device: torch.device
    progress_func: Callable = tqdm_notebook
        
    def __len__(self): return len(self.dl)
    def __iter__(self):
        self.gen = (to_device(self.device,o) for o in self.dl)
        if self.progress_func is not None:
            self.gen = self.progress_func(self.gen, total=len(self.dl), leave=False)
        return iter(self.gen)

    @classmethod
    def create(cls, *args, device=default_device, **kwargs):
        return cls(DataLoader(*args, **kwargs), device=device)

In [238]:
def get_data(train_ds, valid_ds, bs):
    return (DeviceDataLoader.create(train_tds, bs,   shuffle=True),
            DeviceDataLoader.create(valid_tds, bs*2, shuffle=False))

In [239]:
train_dl,valid_dl = get_data(train_tds, valid_tds, bs)

In [240]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2]).to(default_device)
    return model, optim.SGD(model.parameters(), lr=lr)

In [241]:
model,opt = get_model()

In [242]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 1.0774616397857666


1 0.6431661991596221


## Learner

**Define learner**

Finally, we are missing a learner class to close the gap between our loaded data and our model. The learner class will recieve our loaded data (after transformations) and the model and we will be able to call fit on it to start the training phase.

Note that we must define another fit function to track the progress of our training with the progress bar we included in the Dataloader.

In [230]:
#export
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for epoch in tnrange(epochs):
        model.train()
        for xb,yb in train_dl:
            loss,_ = loss_batch(model, xb, yb, loss_fn, opt)
            if train_dl.progress_func is not None: train_dl.gen.set_postfix_str(loss)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [231]:
#export
class DataBunch():
    def __init__(self, train_ds, valid_ds, bs=64, device=None, train_tfm=None, valid_tfm=None):
        self.device = default_device if device is None else device
        self.train_dl = DeviceDataLoader.create(TfmDataset(train_ds,train_tfm), bs, shuffle=True)
        self.valid_dl = DeviceDataLoader.create(TfmDataset(valid_ds, valid_tfm), bs*2, shuffle=False)

class Learner():
    def __init__(self, data, model):
        self.data,self.model = data,model.to(data.device)

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        loss_fn = F.cross_entropy
        fit(epochs, self.model, loss_fn, opt, self.data.train_dl, self.data.valid_dl)

In [232]:
data = DataBunch(train_ds, valid_ds, bs, train_tfm=mnist2image, valid_tfm=mnist2image)
model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)
opt_fn = partial(optim.SGD, momentum=0.9)

In [233]:
learner.fit(4, lr/5, opt_fn=opt_fn)

0 0.54523722448349


1 0.3827536999702454


2 0.27865005321502684


3 0.2523621954917908


In [234]:
learner = Learner(data, simple_cnn([1,16,16,10], [3,3,3], [2,2,2]))

In [235]:
learner.fit(1, lr/5, opt_fn=opt_fn)
learner.fit(2, lr, opt_fn=opt_fn)
learner.fit(1, lr/5, opt_fn=opt_fn)

0 0.44037471532821654


0 0.27522295808792113


1 0.22102980737686156


0 0.16238361406326293


In [38]:
# TODO: metrics